In [2]:
import investpy
import pandas as pd
from fredapi import Fred
import json
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import quandl
from datetime import date
from utils.utils import *
from plotly.subplots import make_subplots
import datetime

with open('keys/keys.json', 'r') as key_file:
    keys = json.load(key_file)
    
with open('utils/index_codes.json', 'r') as indx:
    index_codes = json.load(indx)

In [29]:
class portfolio_analysis:

    def __init__(self, ticker_list, period_type='annual'):
        assert period_type in ['quarterly', 'annual']
        
        self.period_type = period_type
        self.ticker_list = ticker_list
        # year
        currentDateTime = datetime.datetime.now()
        self.date = currentDateTime.date()
        self.year = int(date.strftime("%Y"))

    
    def create_estimates(self, data_type):
        estimate_dict = dict()
        for ticker in self.ticker_list:
            estimate_dict[ticker] = dict()
            estimate_dict[ticker]['data'] = self._seekingAlpha_estimates(ticker, data_type)
            estimate_dict[ticker]['average_growth'] = estimate_dict[ticker]['data']['v_pct_change'].mean()
            estimate_dict[ticker]['growth_type'] = self._determine_growth_type(estimate_dict[ticker]['average_growth'])
        
        self.estimate_dict = estimate_dict
        print("Created `estimate_dict`")
        
    
    def calculate_price_range(self, multiplier=None):
        """
        For stable grower, min max estimation based on diluted eps is fairly accurate.
        However, when it comes to fast grower, market sentiment become relatively more important.
        For slow grower, it refers to divident paying stocks. Thus min max price should be calculated differently.
        """
        if multiplier is None:
            multiplier=[1.5, 2]
        
        for ticker in self.ticker_list:
            next_year_dil_eps = self.estimate_dict[ticker]['data'].loc[\
                                    self.estimate_dict[ticker]['data'].year==self.year+1, 'consensus'].values[0]
            average_growth = self.estimate_dict[ticker]['average_growth']
            self.estimate_dict[ticker]['price_range'] = [next_year_dil_eps*average_growth*m for m in multiplier]
        print("Created price_range")
            
    
    def _determine_growth_type(self, growth):
        if (growth>10)&(growth<20):
            return "stable grower"
        elif growth>=20:
            return "fast grower"
        elif growth<=10:
            return "slow grower"
    
        
    def _seekingAlpha_estimates(self, ticker, data_type, key=keys['rapidAPI_seekingalpha']):
        assert data_type in ['eps', 'revenues']        
        url = "https://seeking-alpha.p.rapidapi.com/symbols/get-estimates"

        querystring = {"symbol": ticker.lower(),
                    "data_type": data_type,
                    "period_type": self.period_type}
        headers = {
            'x-rapidapi-key': key,
            'x-rapidapi-host': "seeking-alpha.p.rapidapi.com"
        }

        response = requests.request("GET", url, headers=headers, params=querystring)
        df = pd.DataFrame([response.json()['data'][i]['attributes'] for i in range(len(response.json()['data']))])
        
        df = df.loc[df.year.isin([self.year-2, self.year-1, self.year, self.year+1])]
        
        df['v'] = df['actual']
        df.loc[df.actual.isna(), 'v'] = df.loc[df.actual.isna(), 'consensus']
        df['v_pct_change'] = df.v.pct_change() * 100

        return df

In [30]:
portfolio = portfolio_analysis(['AAPL', 'AMZN', 'PYPL'])
portfolio.create_estimates('eps')

Created `estimate_dict`


In [31]:
portfolio.calculate_price_range()

Created price range


In [36]:
sum(portfolio.estimate_dict['AAPL']['price_range'])/2

136.42841218836054

152.79

(102.176775, 136.2357)

In [6]:
portfolio.estimate_dict

{'AAPL': {'data':    slug  actual period_end_date  quarter  year  fwd_pe  consensus        v  \
  0  aapl  2.9725      2019-09-30        4  2019   44.83    2.92718  2.97250   
  1  aapl  3.2800      2020-09-30        4  2020   40.55    3.23659  3.28000   
  2  aapl     NaN      2021-09-30        4  2021   29.86    4.39553  4.39553   
  3  aapl     NaN      2022-09-30        4  2022   28.20    4.65453  4.65453   
  4  aapl     NaN      2023-09-30        4  2023   26.02    5.04326  5.04326   
  
     v_pct_change  
  0           NaN  
  1     10.344828  
  2     34.010061  
  3      5.892350  
  4      8.351649  ,
  'average_growth': 14.649721778417453,
  'growth_type': 'stable grower'},
 'AMZN': {'data':    slug  actual period_end_date  quarter  year  fwd_pe  consensus         v  \
  0  amzn   23.01      2019-12-31        4  2019  163.79   20.63258  23.01000   
  1  amzn   41.83      2020-12-31        4  2020   97.16   34.78125  41.83000   
  2  amzn     NaN      2021-12-31        4  20

In [41]:
portfolio.estimate_dict['PYPL'].keys()

dict_keys(['data', 'average_growth', 'growth_type'])

In [48]:
portfolio.estimate_dict['PYPL']['growth_type']

'fast grower'

In [49]:
portfolio.estimate_dict['PYPL']['average_growth']

23.4457187272006

In [53]:
portfolio.estimate_dict['PYPL']['data']

,slug,actual,period_end_date,quarter,year,fwd_pe,consensus,v,v_pct_change
0,pypl,3.10,2019-12-31,4,2019,86.61,3.08017,3.10000,NaN
1,pypl,3.88,2020-12-31,4,2020,70.32,3.79372,3.88000,25.161290
2,pypl,NaN,2021-12-31,4,2021,58.49,4.56117,4.56117,17.555928
3,pypl,NaN,2022-12-31,4,2022,46.44,5.74434,5.74434,25.940055
4,pypl,NaN,2023-12-31,4,2023,37.12,7.18764,7.18764,25.125602


In [56]:
4.5*25.9*1.5, 25.9*2*4.5

(174.825, 233.1)

In [ ]:
4.5

In [50]:
25*23.4457187272006, 33*23.4457187272006

(586.142968180015, 773.7087179976198)

In [ ]:
df = df.drop(['values'], axis=1)

In [21]:
df['v'] = df['actual']
df.loc[df.actual.isna(), 'v'] = df.loc[df.actual.isna(), 'consensus']

df['v_pct_change'] = df.v.pct_change()*100
average_growth = df.v_pct_change.mean()
df['avg_growth'] = average_growth

In [31]:


if (average_growth>10) and (average_growth<=20):
    print("stable_growth")
elif (average_growth>20):
    df['stock_type'] = "fast_growth"
    

stable_growth


In [30]:
df

,slug,actual,period_end_date,quarter,year,fwd_pe,consensus,v,v_pct_change,avg_growth,stock_type
0,aapl,2.9725,2019-09-30,4,2019,45.43,2.92718,2.97250,NaN,14.649722,stable_growth
1,aapl,3.2800,2020-09-30,4,2020,41.09,3.23659,3.28000,10.344828,14.649722,stable_growth
2,aapl,NaN,2021-09-30,4,2021,30.26,4.39553,4.39553,34.010061,14.649722,stable_growth
3,aapl,NaN,2022-09-30,4,2022,28.57,4.65453,4.65453,5.892350,14.649722,stable_growth
4,aapl,NaN,2023-09-30,4,2023,26.37,5.04326,5.04326,8.351649,14.649722,stable_growth


In [ ]:
df['average'] = df['actual']
df.loc[df['actual'].isna(), 'average'] = df.loc[df['actual'].isna(), 'consensus']
df['average'] = df['average'].mean()

In [10]:
df

,slug,actual,period_end_date,quarter,year,fwd_pe,consensus,average
0,aapl,2.9725,2019-09-30,4,2019,45.43,2.92718,4.069164
1,aapl,3.2800,2020-09-30,4,2020,41.09,3.23659,4.069164
2,aapl,NaN,2021-09-30,4,2021,30.26,4.39553,4.069164
3,aapl,NaN,2022-09-30,4,2022,28.57,4.65453,4.069164
4,aapl,NaN,2023-09-30,4,2023,26.37,5.04326,4.069164


In [5]:
def call_estimates(ticker_list, data_type, period_type='annual'):
    estimate_list = list()
    for ticker in ticker_list:
        estimate_list.append(seekingAlpha_estimates(ticker, data_type, period_type))
        
    estimates_all = pd.concat(estimate_list, axis=0)
    return estimates_all

In [7]:
df = call_estimates(["AAPL", "AMZN", "LMT", "MSFT", "PYPL", "O", "GOOGL", "JPM", "PG"], 'eps')

In [8]:
df.head()

,slug,actual,period_end_date,quarter,year,fwd_pe,consensus,average
0,aapl,2.9725,2019-09-30,4,2019,45.43,2.92718,4.067952
1,aapl,3.2800,2020-09-30,4,2020,41.09,3.23659,4.067952
2,aapl,NaN,2021-09-30,4,2021,30.27,4.39318,4.067952
3,aapl,NaN,2022-09-30,4,2022,28.58,4.65396,4.067952
4,aapl,NaN,2023-09-30,4,2023,26.39,5.04012,4.067952
